In [27]:
using ITensors

function ITensors.space(::SiteType"S2=1/2";
    conserve_qns=false)
    if conserve_qns
        return [QN("Sz",1,2)=>1,QN("Sz",0,2)=>1,
                QN("Sz",1,2)=>1,QN("Sz",0,2)=>1,QN("Sz",1,2)=>1]
    end
    return 4
end
ITensors.op(::OpName"Sz",::SiteType"S2=1/2") = [
    3/2 0 0 0 0
    0 -1/2 0 0  0
    0 0 1/2 0 0
    0 0 0 -3/2 0
    0 0 0 0 5/2
]

In [29]:
sites = siteinds("S2=1/2", 4; conserve_qns=true)
ψ = MPS(sites, [1,5,1,1])
println(flux(ψ))
inner(ψ, apply(MPO(AutoMPO() + ("Sz",2), sites), ψ))

QN("Sz",0,2)


2.5

In [64]:

function ITensors.space(
    ::SiteType"PlaRotor";
    dim,
    basis="m",
    conserve_parity=false,
    conserve_inversion_symmetry=false,
    )
    mmax = dim ÷ 2
    is_odd = dim%2
    is_even = 1 - is_odd

    # no conserved quantities
    if !conserve_parity && !conserve_inversion_symmetry
        return dim
    end

    if basis == "m"
        if conserve_inversion_symmetry && conserve_parity
            # all other operators must be transformed with the m_inversion_symmetry operator 
            # then the 
            @assert is_odd == 1
            quantum_numbers = Pair{QN, Int64}[]
            for m = -mmax:mmax
                push!(quantum_numbers, QN(("parity", Int(m>0),2), ("inv_sym", m%2, 2))=>1)
            end
            return quantum_numbers
        elseif conserve_inversion_symmetry
            @assert is_odd == 1
            return [QN("inv_sym",0,2)=>mmax+1, QN("inv_sym",1,2)=>mmax]
        else
            quantum_numbers = Pair{QN, Int64}[]
            for m = -mmax+is_even:mmax
                push!(quantum_numbers, QN("parity", m%2, 2)=>1)
            end
            println(quantum_numbers)
            return quantum_numbers
        end
    elseif basis == "dvr"
        # apply rotation then the inversion operator
        if conserve_inversion_symmetry && conserve_parity
            @assert is_even == 1
            println()
            return [
                QN(("parity",0,2), ("inv_sym",0,2))=>1 + mmax÷2, 
                QN(("parity",0,2), ("inv_sym",1,2))=>(mmax-1)÷2, 
                QN(("parity",1,2), ("inv_sym",0,2))=>(mmax+1)÷2, 
                QN(("parity",1,2), ("inv_sym",1,2))=>mmax÷2, 
            ]
        elseif conserve_inversion_symmetry
            return [QN("parity",0,2)=>mmax+1, QN("parity",1,2)=>mmax-is_even]
        else
            @assert is_even == 1
            return [QN("parity",0,2)=>mmax, QN("parity",1,2)=>mmax]
        end
    else
        throw("Invalid basis: $basis. Must be m or dvr")
    end
end
siteinds("PlaRotor", 4;dim=3,basis="m", conserve_parity=true, conserve_inversion_symmetry=false)

Pair{QN, Int64}[QN("parity",1,2) => 1, QN("parity",0,2) => 1, QN("parity",1,2) => 1]
Pair{QN, Int64}[QN("parity",1,2) => 1, QN("parity",0,2) => 1, QN("parity",1,2) => 1]
Pair{QN, Int64}[QN("parity",1,2) => 1, QN("parity",0,2) => 1, QN("parity",1,2) => 1]
Pair{QN, Int64}[QN("parity",1,2) => 1, QN("parity",0,2) => 1, QN("parity",1,2) => 1]


4-element Vector{Index{Vector{Pair{QN, Int64}}}}:
 (dim=3|id=612|"PlaRotor,Site,n=1") <Out>
 1: QN("parity",1,2) => 1
 2: QN("parity",0,2) => 1
 3: QN("parity",1,2) => 1
 (dim=3|id=417|"PlaRotor,Site,n=2") <Out>
 1: QN("parity",1,2) => 1
 2: QN("parity",0,2) => 1
 3: QN("parity",1,2) => 1
 (dim=3|id=175|"PlaRotor,Site,n=3") <Out>
 1: QN("parity",1,2) => 1
 2: QN("parity",0,2) => 1
 3: QN("parity",1,2) => 1
 (dim=3|id=121|"PlaRotor,Site,n=4") <Out>
 1: QN("parity",1,2) => 1
 2: QN("parity",0,2) => 1
 3: QN("parity",1,2) => 1